In [1]:
%matplotlib inline

from urllib.request import urlretrieve

import matplotlib.pyplot as plt
import nengo
from nengo import spa
import numpy as np
import tensorflow as tf

import nengo_dl

from scipy.signal import decimate

In [2]:
from prophesee.io.psee_loader import PSEELoader

import pathlib
from glob import glob

In [3]:
delta_t = 10000
n = 75
skip = 100

In [4]:
vids = map(PSEELoader, glob("Prophesee_Dataset_n_cars/n-cars_train/cars/*_td.dat"))
# height, width = vid.get_size()
height, width = 120, 120
# vid.seek_time(skip)
event_seqs = [next(vids).load_delta_t(delta_t) for _ in range(n)]
# N = len(event_seqs)

In [5]:
dat = np.zeros((n, delta_t, 30, 30),dtype=np.int8)
for i,block in enumerate(event_seqs):
    for ev in block:
        dat[i,ev[0],ev[1]//4,ev[2]//4] = ev[3]
# for ev in events:
#     dat[ev[0],ev[1],ev[2]] = ev[3]

# dat = decimate(dat,4,axis=2)
# dat = decimate(dat,4,axis=3)

train_data = dat[0:60,:,:,:]
test_data  = dat[60:75,:,:,:]

train_data = train_data.reshape((train_data.shape[0], train_data.shape[1], -1))
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], -1))


In [6]:


with nengo.Network() as auto_net:
    n_in = train_data.shape[2]
    n_hidden=64
    neuron_type = nengo.LIF(amplitude=0.01)
    
    # input
    nengo_a = nengo.Node(np.zeros(n_in))

    # first layer
    nengo_b = nengo.Ensemble(n_hidden, 1, neuron_type=neuron_type)
    nengo.Connection(nengo_a, nengo_b.neurons, transform=nengo_dl.dists.Glorot())

    # second layer
    nengo_c = nengo.Ensemble(n_in, 1, neuron_type=neuron_type)
    nengo.Connection(
        nengo_b.neurons, nengo_c.neurons, transform=nengo_dl.dists.Glorot()
    )

    # probes are used to collect data from the network
    p_c = nengo.Probe(nengo_c.neurons)
    p_b = nengo.Probe(nengo_b.neurons)
    
    

In [7]:
minibatch_size = 8

with nengo_dl.Simulator(network=auto_net, minibatch_size=minibatch_size) as sim:
    out = sim.predict(np.ones((minibatch_size, 1, n_in)))
    print(out[p_c].shape,out[p_b].shape)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


/home/hutch/.pyenv/versions/miniconda3-4.7.12/envs/spiking_encoders/lib/python3.9/site-packages/nengo_dl/simulator.py:1024: UserWarning: Running for one timestep, but the network contains synaptic filters (which will introduce at least a one-timestep delay); did you mean to set synapse=None?
  warnings.warn(


(8, 1, 900) (8, 1, 64)


In [8]:
with nengo_dl.Simulator(auto_net, minibatch_size=minibatch_size) as sim:
    sim.compile(optimizer=tf.optimizers.RMSprop(1e-3), loss=tf.losses.mse)

    # run training loop
    sim.fit(train_data, train_data, epochs=5)

    # evaluate performance on test set
    sim.evaluate(test_data, test_data)


Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


/home/hutch/.pyenv/versions/miniconda3-4.7.12/envs/spiking_encoders/lib/python3.9/site-packages/nengo_dl/simulator.py:1921: UserWarning: Number of elements in input data (60) is not evenly divisible by Simulator.minibatch_size (8); input data will be truncated.
  warnings.warn(
/home/hutch/.pyenv/versions/miniconda3-4.7.12/envs/spiking_encoders/lib/python3.9/site-packages/nengo_dl/simulator.py:1765: UserWarning: Number of elements (1) in ['ndarray'] does not match number of Probes (2); consider using an explicit input dictionary in this case, so that the assignment of data to objects is unambiguous.
  warnings.warn(


Epoch 1/5
7/7 [==============================] - 112s 16s/step - loss: 1.3115 - probe_loss: 1.3115 - probe_1_loss: 0.0000e+00
Epoch 2/5
7/7 [==============================] - 108s 15s/step - loss: 0.3084 - probe_loss: 0.3084 - probe_1_loss: 0.0000e+00
Epoch 3/5
7/7 [==============================] - 106s 15s/step - loss: 0.0582 - probe_loss: 0.0582 - probe_1_loss: 0.0000e+00
Epoch 4/5
7/7 [==============================] - 105s 15s/step - loss: 0.0053 - probe_loss: 0.0053 - probe_1_loss: 0.0000e+00
Epoch 5/5
7/7 [==============================] - 105s 15s/step - loss: 8.2041e-04 - probe_loss: 8.2041e-04 - probe_1_loss: 0.0000e+00
|             Constructing graph: build stage (0%)             | ETA:  --:--:--

/home/hutch/.pyenv/versions/miniconda3-4.7.12/envs/spiking_encoders/lib/python3.9/site-packages/nengo_dl/simulator.py:1921: UserWarning: Number of elements in input data (15) is not evenly divisible by Simulator.minibatch_size (8); input data will be truncated.
  warnings.warn(


1/1 [==============================] - 6s 6s/step - loss: 0.4359 - probe_loss: 0.4359 - probe_1_loss: 0.0000e+00
